In [4]:
#import neceassary packages
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from library.sb_utils import save_file

In [5]:
# load energy and weather datasets into pandas dataframes
energy_data = pd.read_csv('../raw_data/energy_dataset.csv')
weather_data = pd.read_csv('../raw_data/weather_features.csv')

In [6]:
energy_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35064 entries, 0 to 35063
Data columns (total 29 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   time                                         35064 non-null  object 
 1   generation biomass                           35045 non-null  float64
 2   generation fossil brown coal/lignite         35046 non-null  float64
 3   generation fossil coal-derived gas           35046 non-null  float64
 4   generation fossil gas                        35046 non-null  float64
 5   generation fossil hard coal                  35046 non-null  float64
 6   generation fossil oil                        35045 non-null  float64
 7   generation fossil oil shale                  35046 non-null  float64
 8   generation fossil peat                       35046 non-null  float64
 9   generation geothermal                        35046 non-null  float64
 10

In [7]:
energy_data['time'] = pd.to_datetime(energy_data['time'], yearfirst=True, utc=True)

In [8]:
energy_data['generation hydro pumped storage consumption'].describe()

count    35045.000000
mean       475.577343
std        792.406614
min          0.000000
25%          0.000000
50%         68.000000
75%        616.000000
max       4523.000000
Name: generation hydro pumped storage consumption, dtype: float64

In [9]:
energy_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35064 entries, 0 to 35063
Data columns (total 29 columns):
 #   Column                                       Non-Null Count  Dtype              
---  ------                                       --------------  -----              
 0   time                                         35064 non-null  datetime64[ns, UTC]
 1   generation biomass                           35045 non-null  float64            
 2   generation fossil brown coal/lignite         35046 non-null  float64            
 3   generation fossil coal-derived gas           35046 non-null  float64            
 4   generation fossil gas                        35046 non-null  float64            
 5   generation fossil hard coal                  35046 non-null  float64            
 6   generation fossil oil                        35045 non-null  float64            
 7   generation fossil oil shale                  35046 non-null  float64            
 8   generation fossil peat    

In [10]:
#it looks like we have several columns that are completely null. Let's confirm and then delete them:
energy_data['generation fossil coal-derived gas'].sum()

0.0

In [11]:
energy_data['generation fossil oil shale'].sum()

0.0

In [12]:
energy_data['generation fossil peat'].sum()

0.0

In [13]:
energy_data['generation hydro pumped storage aggregated'].count()

0

In [14]:
energy_data['generation geothermal'].sum()

0.0

In [15]:
energy_data['generation marine'].sum()

0.0

In [16]:
energy_data['generation wind offshore'].sum()

0.0

In [17]:
energy_data['forecast wind offshore eday ahead'].count()

0

In [18]:
energy_data.drop(columns = ['generation fossil coal-derived gas','generation fossil oil shale','generation fossil peat',
                            'generation geothermal','generation hydro pumped storage aggregated','generation marine',
                            'generation wind offshore','forecast wind offshore eday ahead'],inplace=True)

In [19]:
#Count (using `.sum()`) the number of missing values (`.isnull()`) in each column of 
#energy_data as well as the percentages (using `.mean()` instead of `.sum()`).
#Order them (increasing or decreasing) using sort_values
#Call `pd.concat` to present these in a single table (DataFrame) with the helpful column names 'count' and '%'
missing = pd.concat([energy_data.isnull().sum(), 100 * energy_data.isnull().mean()], axis=1)
missing.columns=['count', '%']
missing.sort_values(by=['count'],ascending = False)
missing.head(20)


,count,%
time,0,0.000000
generation biomass,19,0.054187
generation fossil brown coal/lignite,18,0.051335
generation fossil gas,18,0.051335
generation fossil hard coal,18,0.051335
generation fossil oil,19,0.054187
generation hydro pumped storage consumption,19,0.054187
generation hydro run-of-river and poundage,19,0.054187
generation hydro water reservoir,18,0.051335
generation nuclear,17,0.048483


The number of null values is very low, but I want to delete as few rows as possible because it would be best to have continuous hourly data. I could impute the missing values with 0 and it would likely not have a significant effect. It is likely that the missing values all come from the same times, since the number of missing values is very similar for each variable. I will need to investigate how best to handle these.

In [20]:
# let's check to see what the null values look like for our target variable:
energy_data.query('`total load actual`.isnull()', engine='python') 

,time,generation biomass,generation fossil brown coal/lignite,generation fossil gas,generation fossil hard coal,generation fossil oil,generation hydro pumped storage consumption,generation hydro run-of-river and poundage,generation hydro water reservoir,generation nuclear,...,generation other renewable,generation solar,generation waste,generation wind onshore,forecast solar day ahead,forecast wind onshore day ahead,total load forecast,total load actual,price day ahead,price actual
108,2015-01-05 11:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3932.0,9258.0,23209.0,NaN,35.50,79.14
109,2015-01-05 12:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4236.0,9156.0,23725.0,NaN,36.80,73.95
110,2015-01-05 13:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4215.0,9072.0,23614.0,NaN,32.50,71.93
111,2015-01-05 14:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4050.0,8779.0,22381.0,NaN,30.00,71.50
112,2015-01-05 15:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3728.0,8426.0,21371.0,NaN,30.00,71.85
113,2015-01-05 16:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3175.0,7946.0,20760.0,NaN,30.60,80.53
661,2015-01-28 12:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4136.0,6223.0,36239.0,NaN,65.00,77.62
751,2015-02-01 06:00:00+00:00,449.0,312.0,4765.0,5269.0,222.0,480.0,980.0,1174.0,7101.0,...,75.0,48.0,208.0,3289.0,18.0,3141.0,24379.0,NaN,56.10,16.98
752,2015-02-01 07:00:00+00:00,453.0,312.0,4938.0,5652.0,288.0,0.0,1031.0,3229.0,7099.0,...,75.0,73.0,207.0,3102.0,63.0,3165.0,27389.0,NaN,57.69,19.56
753,2015-02-01 08:00:00+00:00,452.0,302.0,4997.0,5770.0,296.0,0.0,1083.0,4574.0,7097.0,...,71.0,809.0,204.0,2838.0,691.0,2907.0,30619.0,NaN,60.01,23.13


For the most part, I just have single hours of missing 'total load actual' here and there, except for two instances where there a substantial period of many null vlaues in a row (on 2015-01-05 and 2015-02-01)

In [21]:
energy_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35064 entries, 0 to 35063
Data columns (total 21 columns):
 #   Column                                       Non-Null Count  Dtype              
---  ------                                       --------------  -----              
 0   time                                         35064 non-null  datetime64[ns, UTC]
 1   generation biomass                           35045 non-null  float64            
 2   generation fossil brown coal/lignite         35046 non-null  float64            
 3   generation fossil gas                        35046 non-null  float64            
 4   generation fossil hard coal                  35046 non-null  float64            
 5   generation fossil oil                        35045 non-null  float64            
 6   generation hydro pumped storage consumption  35045 non-null  float64            
 7   generation hydro run-of-river and poundage   35045 non-null  float64            
 8   generation hydro water res

As suspected, the rows that are null are pretty much null across the board. Luckily most of them are not continuous, except for one group of 6 continuous hours on 2015-01-05. Probably the only way to handle these null values is to delete the entire row since it would not be feasible to impute missing values for all the columns of an observation. So if it is a row with all 'generation' columns as null, I will delete the row, but if only a few columns are null, I will impute missing vlaues with 0. But it is maybe best not to delete these until we merge with the weather data.

In [22]:
weather_data.info
#we can see that the times seem to match up, so if I want to merge the two DF it will be on 'time' = 'dt_iso',
#however, there is only one instance of each time in the energy data and one for each city (x5) in the weather data
#It also appears that there is indeed one observation for each hour over four years. 
#365*3*24 + 366*24 = 35064 (for one leap year). Somehow the weather data has more entries than that though. 
# 5 cities * 35064 = 175320, but there are 178395 rows. I will have to invesigate where the ~3000 extra rows come from

<bound method DataFrame.info of                            dt_iso city_name     temp  temp_min  temp_max  \
0       2015-01-01 00:00:00+01:00  Valencia  270.475   270.475   270.475   
1       2015-01-01 01:00:00+01:00  Valencia  270.475   270.475   270.475   
2       2015-01-01 02:00:00+01:00  Valencia  269.686   269.686   269.686   
3       2015-01-01 03:00:00+01:00  Valencia  269.686   269.686   269.686   
4       2015-01-01 04:00:00+01:00  Valencia  269.686   269.686   269.686   
...                           ...       ...      ...       ...       ...   
178391  2018-12-31 19:00:00+01:00   Seville  287.760   287.150   288.150   
178392  2018-12-31 20:00:00+01:00   Seville  285.760   285.150   286.150   
178393  2018-12-31 21:00:00+01:00   Seville  285.150   285.150   285.150   
178394  2018-12-31 22:00:00+01:00   Seville  284.150   284.150   284.150   
178395  2018-12-31 23:00:00+01:00   Seville  283.970   282.150   285.150   

        pressure  humidity  wind_speed  wind_deg  rain_

In [23]:
weather_data[weather_data.city_name == 'Valencia'].count()

dt_iso                 35145
city_name              35145
temp                   35145
temp_min               35145
temp_max               35145
pressure               35145
humidity               35145
wind_speed             35145
wind_deg               35145
rain_1h                35145
rain_3h                35145
snow_3h                35145
clouds_all             35145
weather_id             35145
weather_main           35145
weather_description    35145
weather_icon           35145
dtype: int64

In [24]:
weather_data[weather_data.city_name == 'Madrid'].count()

dt_iso                 36267
city_name              36267
temp                   36267
temp_min               36267
temp_max               36267
pressure               36267
humidity               36267
wind_speed             36267
wind_deg               36267
rain_1h                36267
rain_3h                36267
snow_3h                36267
clouds_all             36267
weather_id             36267
weather_main           36267
weather_description    36267
weather_icon           36267
dtype: int64

In [25]:
weather_data[weather_data.city_name == 'Bilbao'].count()

dt_iso                 35951
city_name              35951
temp                   35951
temp_min               35951
temp_max               35951
pressure               35951
humidity               35951
wind_speed             35951
wind_deg               35951
rain_1h                35951
rain_3h                35951
snow_3h                35951
clouds_all             35951
weather_id             35951
weather_main           35951
weather_description    35951
weather_icon           35951
dtype: int64

In [26]:
weather_data[weather_data.city_name == ' Barcelona'].count() #noticed that Barecelona has a space in front

dt_iso                 35476
city_name              35476
temp                   35476
temp_min               35476
temp_max               35476
pressure               35476
humidity               35476
wind_speed             35476
wind_deg               35476
rain_1h                35476
rain_3h                35476
snow_3h                35476
clouds_all             35476
weather_id             35476
weather_main           35476
weather_description    35476
weather_icon           35476
dtype: int64

In [27]:
weather_data[weather_data.city_name == 'Seville'].count()

dt_iso                 35557
city_name              35557
temp                   35557
temp_min               35557
temp_max               35557
pressure               35557
humidity               35557
wind_speed             35557
wind_deg               35557
rain_1h                35557
rain_3h                35557
snow_3h                35557
clouds_all             35557
weather_id             35557
weather_main           35557
weather_description    35557
weather_icon           35557
dtype: int64

In [28]:
#delete the space in the Barcelona city name before I forget
weather_data.replace(to_replace =' Barcelona', value ='Barcelona', inplace = True)

In [29]:
weather_data['dt_iso'].value_counts()

2015-10-01 02:00:00+02:00    10
2017-09-30 02:00:00+02:00    10
2016-09-30 02:00:00+02:00    10
2018-09-30 02:00:00+02:00    10
2018-02-28 09:00:00+01:00    10
                             ..
2016-05-06 17:00:00+02:00     5
2016-05-06 15:00:00+02:00     5
2016-05-06 14:00:00+02:00     5
2016-05-06 13:00:00+02:00     5
2018-12-31 23:00:00+01:00     5
Name: dt_iso, Length: 35064, dtype: int64

So there are many actual duplicates (not just the 5x duplicates indicated by having 5 cities). We will eventually need to transform the weather data to be wide (have only one time observation per hour and make the city data be columns).

In [30]:
weather_data.drop_duplicates(inplace = True) 

In [31]:
weather_data['dt_iso'].value_counts()

2018-02-28 09:00:00+01:00    10
2018-02-28 10:00:00+01:00     9
2018-01-07 10:00:00+01:00     9
2018-02-28 11:00:00+01:00     9
2017-02-24 11:00:00+01:00     9
                             ..
2016-05-06 15:00:00+02:00     5
2016-05-06 14:00:00+02:00     5
2016-05-06 13:00:00+02:00     5
2016-05-06 12:00:00+02:00     5
2018-12-31 23:00:00+01:00     5
Name: dt_iso, Length: 35064, dtype: int64

In [32]:
#dropping duplicates did not solve the issue so they must not be duplicates in every field
#let's see what one of the duplicate time values rows look like:
weather_data[weather_data.dt_iso == '2018-02-28 09:00:00+01:00']

,dt_iso,city_name,temp,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,rain_1h,rain_3h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon
27768,2018-02-28 09:00:00+01:00,Valencia,279.15,279.15,279.15,1002,100,1,310,0.0,0.0,0.0,75,803,clouds,broken clouds,04d
63615,2018-02-28 09:00:00+01:00,Madrid,278.15,277.15,279.15,994,93,7,205,0.9,0.0,0.0,90,501,rain,moderate rain,10d
63616,2018-02-28 09:00:00+01:00,Madrid,278.15,277.15,279.15,994,93,7,205,0.9,0.0,0.0,90,701,mist,mist,50d
63617,2018-02-28 09:00:00+01:00,Madrid,278.15,277.15,279.15,994,93,7,205,0.9,0.0,0.0,90,301,drizzle,drizzle,09d
63618,2018-02-28 09:00:00+01:00,Madrid,278.15,277.15,279.15,994,93,7,205,0.9,0.0,0.0,90,741,fog,fog,50d
99837,2018-02-28 09:00:00+01:00,Bilbao,271.15,271.15,271.15,998,100,1,30,0.0,0.0,0.0,90,601,snow,snow,13d
99838,2018-02-28 09:00:00+01:00,Bilbao,271.15,271.15,271.15,998,100,1,30,0.0,0.0,0.0,90,741,fog,fog,50d
135320,2018-02-28 09:00:00+01:00,Barcelona,274.15,273.15,275.15,1006,100,2,30,0.3,0.0,0.0,75,601,snow,snow,13d
135321,2018-02-28 09:00:00+01:00,Barcelona,274.15,273.15,275.15,1006,100,2,30,0.3,0.0,0.0,75,500,rain,light rain,10d
170891,2018-02-28 09:00:00+01:00,Seville,286.15,286.15,286.15,999,93,7,190,0.9,0.0,0.0,90,501,rain,moderate rain,10d


So we have instances where there are multiple hourly observations for the same city, but with different descriptors, id, icon, etc. I'd like to delete all but the first hourly observation for each city. 

In [33]:
#I suspect that temp, temp_min, and temp_max are all equal, let's check and delete the two extra columns if so.
weather_data["temp"].equals(weather_data["temp_min"])
#Okay, columns are not always the same, but since the data are hourly I assume it will be safe to delete 
# temp_max and temp_min and just use temp

False

In [34]:
weather_data["weather_description"].unique()
#These seem too detailed and too numerous to be of use; this column should likely be deleted

array(['sky is clear', 'few clouds', 'scattered clouds', 'broken clouds',
       'overcast clouds', 'light rain', 'moderate rain',
       'heavy intensity rain', 'mist', 'heavy intensity shower rain',
       'shower rain', 'very heavy rain', 'thunderstorm with heavy rain',
       'thunderstorm with light rain', 'thunderstorm with rain',
       'proximity thunderstorm', 'thunderstorm',
       'light intensity shower rain', 'light intensity drizzle', 'fog',
       'drizzle', 'smoke', 'heavy intensity drizzle', 'haze',
       'proximity shower rain', 'light intensity drizzle rain',
       'light snow', 'rain and snow', 'light rain and snow', 'snow',
       'light thunderstorm', 'heavy snow', 'sleet', 'rain and drizzle',
       'shower sleet', 'light shower sleet', 'light shower snow',
       'proximity moderate rain', 'ragged shower rain',
       'sand dust whirls', 'proximity drizzle', 'dust', 'squalls'],
      dtype=object)

In [35]:
weather_data["weather_id"].unique()
# these are codes that go along with the descriptors above

array([800, 801, 802, 803, 804, 500, 501, 502, 701, 522, 521, 503, 202,
       200, 201, 211, 520, 300, 741, 301, 711, 302, 721, 310, 600, 616,
       615, 601, 210, 602, 611, 311, 612, 620, 531, 731, 761, 771],
      dtype=int64)

In [36]:
weather_data["weather_main"].unique()
#This variable seems like it is basic enough that it would be worth quantifying to make a numerical categorical variable

array(['clear', 'clouds', 'rain', 'mist', 'thunderstorm', 'drizzle',
       'fog', 'smoke', 'haze', 'snow', 'dust', 'squall'], dtype=object)

In [37]:
weather_data["weather_icon"].unique()
#these are codes for the weather icon (or thumbnail picture) to describe weather (ie picture of sun or clouds, etc)

array(['01n', '01d', '1', '02n', '02d', '2', '3', '04n', '4', '10n',
       '03n', '10', '04d', '03d', '10d', '50d', '09n', '11d', '11n',
       '09d', '50n', '13d', '13n', '13'], dtype=object)

In [38]:
#let's have a more detailed look at the rain, snow, and clouds features
weather_data[['rain_1h','rain_3h', 'snow_3h', 'clouds_all']].describe()

,rain_1h,rain_3h,snow_3h,clouds_all
count,178375.000000,178375.000000,178375.000000,178375.000000
mean,0.075499,0.000379,0.004763,25.075302
std,0.398870,0.007285,0.222617,30.774908
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,20.000000
75%,0.000000,0.000000,0.000000,40.000000
max,12.000000,2.315000,21.500000,100.000000


It appears the rain_3h mean and max are less than the rain_1h mean and max, which shouldn't be true, so there may be something funky going on. And in any case, 1hr rainfall data should be sufficient. So let's delete the 3h column. I'll also delete the 'weather_id','weather_icon','weather_description','temp_min', and 'temp_max' columns.

In [39]:
#I suspect I will also not need rain_1hr or snow_3hr, and maybe not wind_deg, but I will keep them for now.
weather_data.drop(columns = ['rain_3h', 'weather_id','weather_icon','weather_description','temp_min','temp_max'],inplace = True)

In [40]:
#let's delete duplicate rows now:
weather_data.drop_duplicates(subset = ['dt_iso','city_name'], keep='first', inplace=True)

In [41]:
weather_data[weather_data.city_name == 'Valencia'].count()

dt_iso          35064
city_name       35064
temp            35064
pressure        35064
humidity        35064
wind_speed      35064
wind_deg        35064
rain_1h         35064
snow_3h         35064
clouds_all      35064
weather_main    35064
dtype: int64

In [42]:
weather_data[weather_data.city_name == 'Madrid'].count()

dt_iso          35064
city_name       35064
temp            35064
pressure        35064
humidity        35064
wind_speed      35064
wind_deg        35064
rain_1h         35064
snow_3h         35064
clouds_all      35064
weather_main    35064
dtype: int64

In [43]:
weather_data[weather_data.city_name == 'Barcelona'].count()

dt_iso          35064
city_name       35064
temp            35064
pressure        35064
humidity        35064
wind_speed      35064
wind_deg        35064
rain_1h         35064
snow_3h         35064
clouds_all      35064
weather_main    35064
dtype: int64

In [44]:
weather_data[weather_data.city_name == 'Bilbao'].count()

dt_iso          35064
city_name       35064
temp            35064
pressure        35064
humidity        35064
wind_speed      35064
wind_deg        35064
rain_1h         35064
snow_3h         35064
clouds_all      35064
weather_main    35064
dtype: int64

In [45]:
weather_data[weather_data.city_name == 'Seville'].count()

dt_iso          35064
city_name       35064
temp            35064
pressure        35064
humidity        35064
wind_speed      35064
wind_deg        35064
rain_1h         35064
snow_3h         35064
clouds_all      35064
weather_main    35064
dtype: int64

Excellent, now there is only one hourly observation for each city. Now I have to determine what would be the best way to merge the energy and weather dataframes. If I just average the columns in the weather data for all five cities I might lose some meaningful information, but there would be only one neat set of weather variables. I could also create a wide dataframe with columns for each of the weather variables for each city, but this may create too many variables. Not sure which approach is better...

In [51]:
# create a df with averages for each column (drop city_name column)
weather_average = weather_data.drop(columns = ['city_name','weather_main', 'wind_deg']) #deleting wind deg also because it is
                                                                            #categorical but not useful enough to encode
#create dict for weather_main:numeric value
#weather_dict = {'clear':0, 'clouds':1, 'rain':2, 'mist':3, 'thunderstorm':4, 'drizzle':5, 
#                'fog':6, 'smoke':7, 'haze':8, 'snow':9, 'dust':10, 'squall':11}

# categorize the weather_main column 
#weather_avg['weather_main'].replace(weather_dict, inplace = True)

#making sure the value remap worked:
#weather_avg['weather_main'].value_counts()

# average all other columns grouped by time
weather_avg = weather_average.groupby(['dt_iso'],as_index = False).mean()  #categorical 'weather_main' average will be distorted, take floor int value to convert back to category? 
weather_avg.head()


,dt_iso,temp,pressure,humidity,wind_speed,rain_1h,snow_3h,clouds_all
0,2015-01-01 00:00:00+01:00,272.491463,1016.4,82.4,2.0,0.0,0.0,0.0
1,2015-01-01 01:00:00+01:00,272.512700,1016.2,82.4,2.0,0.0,0.0,0.0
2,2015-01-01 02:00:00+01:00,272.099137,1016.8,82.0,2.4,0.0,0.0,0.0
3,2015-01-01 03:00:00+01:00,272.089469,1016.6,82.0,2.4,0.0,0.0,0.0
4,2015-01-01 04:00:00+01:00,272.145900,1016.6,82.0,2.4,0.0,0.0,0.0


Excellent, it appears it worked to take the averages of the variables for the 5 cities to have only one observation per hour! I'm not sure what the best method for encoding the nominal weather_main variable is. After first Label Encoding them, I have decided for now to drop this column from the averaged df. It seems there are too many categories to do One-Hot encoding and label encoding would not be appropriate for a nominal variable. For now I will try working without this variable, but I could do some feature engineering later to create an ordinal categorical variable with fewer categories.

In [52]:
#create a wide df with the five cities converted to columns and only one time observation per hour

weather_wide = weather_data.pivot(index = 'dt_iso',columns = 'city_name', values = ['temp','pressure','humidity','wind_speed',
'wind_deg','rain_1h','snow_3h','clouds_all','weather_main'])
weather_wide.head()

temp                                         \
city_name                 Barcelona      Bilbao   Madrid  Seville Valencia   
dt_iso                                                                       
2015-01-01 00:00:00+01:00   281.625  269.657312  267.325  273.375  270.475   
2015-01-01 01:00:00+01:00   281.625    269.7635  267.325  273.375  270.475   
2015-01-01 02:00:00+01:00   281.286  269.251688  266.186  274.086  269.686   
2015-01-01 03:00:00+01:00   281.286  269.203344  266.186  274.086  269.686   
2015-01-01 04:00:00+01:00   281.286    269.4855  266.186  274.086  269.686   

                           pressure                                 ...  \
city_name                 Barcelona Bilbao Madrid Seville Valencia  ...   
dt_iso                                                              ...   
2015-01-01 00:00:00+01:00      1035   1036    971    1039     1001  ...   
2015-01-01 01:00:00+01:00      1035   1035    971    1039     1001  ...   
2015-01-01 02:00:00+01:00      1036   1036    971    1039     1002  ...   
2015-01-01 03:00:00+01:00      1036   1035    971    1039     1002  ...   
2015-01-01 04:00:00+01:00      1036   1035    971    1039     1002  ...   

                          clouds_all                                 \
city_name                  Barcelona Bilbao Madrid Seville Valencia   
dt_iso                                                                
2015-01-01 00:00:00+01:00          0      0      0       0        0   
2015-01-01 01:00:00+01:00          0      0      0       0        0   
2015-01-01 02:00:00+01:00          0      0      0       0        0   
2015-01-01 03:00:00+01:00          0      0      0       0        0   
2015-01-01 04:00:00+01:00          0      0      0       0        0   

                          weather_main                                 
city_name                    Barcelona Bilbao Madrid Seville Valencia  
dt_iso                                                                 
2015-01-01 00:00:00+01:00        clear  clear  clear   clear    clear  
2015-01-01 01:00:00+01:00        clear  clear  clear   clear    clear  
2015-01-01 02:00:00+01:00        clear  clear  clear   clear    clear  
2015-01-01 03:00:00+01:00        clear  clear  clear   clear    clear  
2015-01-01 04:00:00+01:00        clear  clear  clear   clear    clear  

[5 rows x 45 columns]

In [53]:
weather_wide.reset_index(inplace = True)
weather_wide.head(2)

dt_iso      temp                                \
city_name                            Barcelona      Bilbao   Madrid  Seville   
0          2015-01-01 00:00:00+01:00   281.625  269.657312  267.325  273.375   
1          2015-01-01 01:00:00+01:00   281.625    269.7635  267.325  273.375   

                    pressure                        ... clouds_all         \
city_name Valencia Barcelona Bilbao Madrid Seville  ...  Barcelona Bilbao   
0          270.475      1035   1036    971    1039  ...          0      0   
1          270.475      1035   1035    971    1039  ...          0      0   

                                  weather_main                                 
city_name Madrid Seville Valencia    Barcelona Bilbao Madrid Seville Valencia  
0              0       0        0        clear  clear  clear   clear    clear  
1              0       0        0        clear  clear  clear   clear    clear  

[2 rows x 46 columns]

In [54]:
weather_wide.columns = weather_wide.columns.map('_'.join)
weather_wide.head()

,dt_iso_,temp_Barcelona,temp_Bilbao,temp_Madrid,temp_Seville,temp_Valencia,pressure_Barcelona,pressure_Bilbao,pressure_Madrid,pressure_Seville,...,clouds_all_Barcelona,clouds_all_Bilbao,clouds_all_Madrid,clouds_all_Seville,clouds_all_Valencia,weather_main_Barcelona,weather_main_Bilbao,weather_main_Madrid,weather_main_Seville,weather_main_Valencia
0,2015-01-01 00:00:00+01:00,281.625,269.657312,267.325,273.375,270.475,1035,1036,971,1039,...,0,0,0,0,0,clear,clear,clear,clear,clear
1,2015-01-01 01:00:00+01:00,281.625,269.7635,267.325,273.375,270.475,1035,1035,971,1039,...,0,0,0,0,0,clear,clear,clear,clear,clear
2,2015-01-01 02:00:00+01:00,281.286,269.251688,266.186,274.086,269.686,1036,1036,971,1039,...,0,0,0,0,0,clear,clear,clear,clear,clear
3,2015-01-01 03:00:00+01:00,281.286,269.203344,266.186,274.086,269.686,1036,1035,971,1039,...,0,0,0,0,0,clear,clear,clear,clear,clear
4,2015-01-01 04:00:00+01:00,281.286,269.4855,266.186,274.086,269.686,1036,1035,971,1039,...,0,0,0,0,0,clear,clear,clear,clear,clear


In [55]:
weather_wide.drop(columns = ['weather_main_Barcelona','weather_main_Bilbao','weather_main_Madrid','weather_main_Seville','weather_main_Valencia'],inplace = True)
weather_wide.head()

,dt_iso_,temp_Barcelona,temp_Bilbao,temp_Madrid,temp_Seville,temp_Valencia,pressure_Barcelona,pressure_Bilbao,pressure_Madrid,pressure_Seville,...,snow_3h_Barcelona,snow_3h_Bilbao,snow_3h_Madrid,snow_3h_Seville,snow_3h_Valencia,clouds_all_Barcelona,clouds_all_Bilbao,clouds_all_Madrid,clouds_all_Seville,clouds_all_Valencia
0,2015-01-01 00:00:00+01:00,281.625,269.657312,267.325,273.375,270.475,1035,1036,971,1039,...,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0
1,2015-01-01 01:00:00+01:00,281.625,269.7635,267.325,273.375,270.475,1035,1035,971,1039,...,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0
2,2015-01-01 02:00:00+01:00,281.286,269.251688,266.186,274.086,269.686,1036,1036,971,1039,...,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0
3,2015-01-01 03:00:00+01:00,281.286,269.203344,266.186,274.086,269.686,1036,1035,971,1039,...,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0
4,2015-01-01 04:00:00+01:00,281.286,269.4855,266.186,274.086,269.686,1036,1035,971,1039,...,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0


In [56]:
# save the data to a new csv file
datapath = '../raw_data'
save_file(weather_wide, 'weather_wide.csv', datapath)

A file already exists with this name.

Do you want to overwrite? (Y/N)n

Please re-run this cell with a new filename.


In [57]:
# save the data to a new csv file
datapath = '../raw_data'
save_file(weather_avg, 'weather_avg.csv', datapath)

A file already exists with this name.

Do you want to overwrite? (Y/N)y
Writing file.  "../raw_data\weather_avg.csv"


In [67]:
datapath = '../raw_data'
save_file(energy_data, 'energy_data.csv', datapath)

A file already exists with this name.

Do you want to overwrite? (Y/N)y
Writing file.  "../raw_data\energy_data.csv"
